## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Qaanaaq,GL,77.4840,-69.3632,0.43,66,66,7.81,broken clouds,2021-12-04 17:17:44
1,1,Mataura,NZ,-46.1927,168.8643,58.48,93,58,3.00,broken clouds,2021-12-04 17:17:45
2,2,Fort Nelson,CA,58.8053,-122.7002,-18.81,76,90,2.30,light snow,2021-12-04 17:17:45
3,3,Bathsheba,BB,13.2167,-59.5167,86.61,58,40,19.57,scattered clouds,2021-12-04 17:17:45
4,4,Ribeira Grande,PT,38.5167,-28.7000,63.00,94,75,4.61,light intensity drizzle,2021-12-04 17:17:46


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp']>=min_temp)]

preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
3,3,Bathsheba,BB,13.2167,-59.5167,86.61,58,40,19.57,scattered clouds,2021-12-04 17:17:45
6,6,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,54,40,10.36,scattered clouds,2021-12-04 17:17:46
12,12,Kavaratti,IN,10.5669,72.6420,81.81,75,11,5.64,few clouds,2021-12-04 17:17:48
15,15,Itacare,BR,-14.2775,-38.9967,84.09,69,42,7.90,scattered clouds,2021-12-04 17:17:49
22,22,Dzaoudzi,YT,-12.7887,45.2699,82.29,78,0,4.61,clear sky,2021-12-04 17:17:52
...,...,...,...,...,...,...,...,...,...,...,...
661,661,Tanete,ID,-3.9443,119.7848,75.58,91,100,3.09,overcast clouds,2021-12-04 17:35:23
663,663,Beloha,MG,-25.1667,45.0500,77.49,62,0,5.64,clear sky,2021-12-04 17:35:23
667,667,Opuwo,NaN,-18.0607,13.8400,77.18,26,57,13.53,broken clouds,2021-12-04 17:35:25
670,670,Gurupi,BR,-11.7292,-49.0686,83.48,71,100,4.94,moderate rain,2021-12-04 17:35:26


In [5]:
len(preferred_cities_df)

198

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.eq('').sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

len(clean_df)

196

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,
6,Arraial Do Cabo,BR,80.56,scattered clouds,-22.9661,-42.0278,
12,Kavaratti,IN,81.81,few clouds,10.5669,72.6420,
15,Itacare,BR,84.09,scattered clouds,-14.2775,-38.9967,
22,Dzaoudzi,YT,82.29,clear sky,-12.7887,45.2699,
30,Hambantota,LK,78.87,broken clouds,6.1241,81.1185,
31,Vaini,TO,78.96,moderate rain,-21.2000,-175.2000,
41,Atuona,PF,78.28,scattered clouds,-9.8000,-139.0333,
42,Rikitea,PF,75.42,broken clouds,-23.1203,-134.9692,
44,Castro,BR,79.86,clear sky,-24.7911,-50.0119,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not found...skipping.')


Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,Atlantis Hotel
6,Arraial Do Cabo,BR,80.56,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
12,Kavaratti,IN,81.81,few clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
15,Itacare,BR,84.09,scattered clouds,-14.2775,-38.9967,Ecoporan Hotel Charme Spa & Eventos
22,Dzaoudzi,YT,82.29,clear sky,-12.7887,45.2699,Le Rocher
...,...,...,...,...,...,...,...
657,Bull Savanna,JM,81.30,overcast clouds,17.8868,-77.5902,Big Red's
661,Tanete,ID,75.58,overcast clouds,-3.9443,119.7848,Wisma Tri Multi
663,Beloha,MG,77.49,clear sky,-25.1667,45.0500,
670,Gurupi,BR,83.48,moderate rain,-11.7292,-49.0686,Via Norte Hotel


In [19]:
hotel_df = hotel_df.replace("",np.nan)
hotel_df[hotel_df['Hotel Name'].isna()]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
86,Orange Walk,BZ,82.26,broken clouds,17.8333,-88.8333,NaN
195,Ngama,TD,86.27,clear sky,11.7833,17.1667,NaN
238,Tanout,NE,80.22,overcast clouds,14.9709,8.8879,NaN
322,Adrar,MR,78.96,broken clouds,20.5022,-10.0711,NaN
455,Damara,CF,84.83,broken clouds,4.9608,18.7035,NaN
594,Aitape,PG,78.67,moderate rain,-3.1374,142.3475,NaN
596,Oum Hadjer,TD,85.15,clear sky,13.2954,19.6966,NaN
608,Sembe,CG,76.50,light rain,1.6481,14.5806,NaN
626,Kilimatinde,TZ,76.71,broken clouds,-5.8500,34.9500,NaN
663,Beloha,MG,77.49,clear sky,-25.1667,45.0500,NaN


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0)

In [24]:
clean_hotel_df[clean_hotel_df['Hotel Name'].isna()]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [25]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,Atlantis Hotel
6,Arraial Do Cabo,BR,80.56,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
12,Kavaratti,IN,81.81,few clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
15,Itacare,BR,84.09,scattered clouds,-14.2775,-38.9967,Ecoporan Hotel Charme Spa & Eventos
22,Dzaoudzi,YT,82.29,clear sky,-12.7887,45.2699,Le Rocher


In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} &degF'</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
# heatmap of temperature
# get the latitude and longitude
locations = clean_hotel_df[['Lat','Lng']]
#get the max temperature
max_temp = clean_hotel_df['Max Temp']
# assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=300,
                                point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# add the heatmap layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# call the figure to plot the data
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))